In [6]:
import pandas as pd
import os

In [61]:
test = pd.read_csv(r'D:\Test\trips\2022-09-22-09-31-41-R.csv')

In [85]:
filelist = os.listdir(r'D:\Test')
filelist_ = []
for f in filelist:
    if len(f.split('.')) > 1:
        if f.split('.')[1] == 'pcap':
            filelist_.append(f)

In [87]:
date = [f.split('.')[0] for f in filelist_]
date_ = []
for d in date:
    if d[-1] == 'R':
        date_.append(d[:-2])
    else:
        date_.append(d)
date_ = pd.to_datetime(pd.Series(date_),format=('%Y-%m-%d-%H-%M-%S'))

In [88]:
Ts_records = pd.to_datetime(test.Timestamp,format=('%Y-%m-%d %H:%M:%S'))

In [94]:
date_

0   2022-09-22 09:31:41
1   2022-09-22 12:30:00
2   2022-09-22 16:00:00
dtype: datetime64[ns]

In [95]:
Ts_records

0        2022-09-22 08:31:43.041325092
1        2022-09-22 08:31:43.141324997
2        2022-09-22 08:31:43.241325140
3        2022-09-22 08:31:43.341325045
4        2022-09-22 08:31:43.441325188
                      ...             
632620   2022-09-22 13:14:38.041325092
632621   2022-09-22 13:14:38.141324997
632622   2022-09-22 13:14:38.241325140
632623   2022-09-22 13:14:40.041325092
632624   2022-09-22 13:14:40.041325092
Name: Timestamp, Length: 632625, dtype: datetime64[ns]

In [91]:
(Ts_records.iloc[1] - date_)

0   -1 days +23:00:02.141324997
1   -1 days +20:01:43.141324997
2   -1 days +16:31:43.141324997
dtype: timedelta64[ns]